In [2]:
pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [4]:
df=pd.read_csv(r"D:\Machine learning\Machine-Learning-Projects-Showcase-\AdvancedML & Feature Drift\data\loan_data.csv")

In [5]:
df

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,27.0,male,Associate,47971.0,6,RENT,15000.0,MEDICAL,15.66,0.31,3.0,645,No,1
44996,37.0,female,Associate,65800.0,17,RENT,9000.0,HOMEIMPROVEMENT,14.07,0.14,11.0,621,No,1
44997,33.0,male,Associate,56942.0,7,RENT,2771.0,DEBTCONSOLIDATION,10.02,0.05,10.0,668,No,1
44998,29.0,male,Bachelor,33164.0,4,RENT,12000.0,EDUCATION,13.23,0.36,6.0,604,No,1


In [6]:
# Load your data
df=pd.read_csv(r"D:\Machine learning\Machine-Learning-Projects-Showcase-\AdvancedML & Feature Drift\data\loan_data.csv")  # replace with your path

# Separate target
X = df.drop("loan_status", axis=1)
y = df["loan_status"]

# Column categories
#categorical_cols = X.select_dtypes(include="object").columns.tolist()
#numerical_cols = X.select_dtypes(exclude="object").columns.tolist()

# ONLY done before pipeline creation, during training
categorical_cols = [
    "person_gender", "person_education", "person_home_ownership",
    "loan_intent", "previous_loan_defaults_on_file"
]

numerical_cols = [
    "person_age", "person_income", "person_emp_exp",
    "loan_amnt", "loan_int_rate", "loan_percent_income",
    "cb_person_cred_hist_length", "credit_score"
]


# Split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Define transformers
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical_cols),
    ("cat", OneHotEncoder(drop="first", handle_unknown='ignore'), categorical_cols)
])

# Create pipeline
pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("model", LogisticRegression())
])

# Train
pipeline.fit(x_train, y_train)

# Save full pipeline (includes scaler + encoder + model)
save_path = r"D:\Machine learning\Machine-Learning-Projects-Showcase-\AdvancedML & Feature Drift\deploy\deployment"
os.makedirs(save_path, exist_ok=True)
joblib.dump(pipeline, os.path.join(save_path, "loan_pipeline.pkl"))


['D:\\Machine learning\\Machine-Learning-Projects-Showcase-\\AdvancedML & Feature Drift\\deploy\\deployment\\loan_pipeline.pkl']

In [7]:
pipeline.named_steps['preprocess'].get_feature_names_out()

array(['num__person_age', 'num__person_income', 'num__person_emp_exp',
       'num__loan_amnt', 'num__loan_int_rate', 'num__loan_percent_income',
       'num__cb_person_cred_hist_length', 'num__credit_score',
       'cat__person_gender_male', 'cat__person_education_Bachelor',
       'cat__person_education_Doctorate',
       'cat__person_education_High School',
       'cat__person_education_Master', 'cat__person_home_ownership_OTHER',
       'cat__person_home_ownership_OWN',
       'cat__person_home_ownership_RENT', 'cat__loan_intent_EDUCATION',
       'cat__loan_intent_HOMEIMPROVEMENT', 'cat__loan_intent_MEDICAL',
       'cat__loan_intent_PERSONAL', 'cat__loan_intent_VENTURE',
       'cat__previous_loan_defaults_on_file_Yes'], dtype=object)

In [8]:
a=df.iloc[0]   # geting input festure name

In [9]:
b = pd.DataFrame([a])
b
a    # geting input festure name

person_age                            22.0
person_gender                       female
person_education                    Master
person_income                      71948.0
person_emp_exp                           0
person_home_ownership                 RENT
loan_amnt                          35000.0
loan_intent                       PERSONAL
loan_int_rate                        16.02
loan_percent_income                   0.49
cb_person_cred_hist_length             3.0
credit_score                           561
previous_loan_defaults_on_file          No
loan_status                              1
Name: 0, dtype: object

In [10]:
import requests
from bs4 import BeautifulSoup

url = "http://127.0.0.1:5000/predict"

In [14]:


sample_input = {
    "person_age": 29.0,
    "person_gender": "male",
    "person_education": "Master",
    "person_income": 71948.0,
    "person_emp_exp": 0,
    "person_home_ownership": "RENT",
    "loan_amnt": 35000.0,
    "loan_intent": "PERSONAL",
    "loan_int_rate": 16.02,
    "loan_percent_income": 0.49,
    "cb_person_cred_hist_length": 3.0,
    "credit_score": 561,
    "previous_loan_defaults_on_file": "Yes"
}

response = requests.post(url, data=sample_input)

soup = BeautifulSoup(response.text, "html.parser")
prediction = soup.find("h3")  # or whatever tag your HTML uses
print("Prediction from Web App:", prediction.text if prediction else "Not Found")


Prediction from Web App: 🧠 Prediction Result: 0


In [16]:
# pred : 1
sample_input = {
  "person_age": 22.0,
  "person_gender": "female",
  "person_education": "Master",
  "person_income": 71948.0,
  "person_emp_exp": 0,
  "person_home_ownership": "RENT",
  "loan_amnt": 35000.0,
  "loan_intent": "PERSONAL",
  "loan_int_rate": 16.02,
  "loan_percent_income": 0.49,
  "cb_person_cred_hist_length": 3.0,
  "credit_score": 561,
  "previous_loan_defaults_on_file": "No"
}

response = requests.post(url, data=sample_input)

soup = BeautifulSoup(response.text, "html.parser")
prediction = soup.find("h3")  # or whatever tag your HTML uses
print("Prediction from Web App:", prediction.text if prediction else "Not Found")

Prediction from Web App: 🧠 Prediction Result: 1
